In [186]:
import pandas as pd 
import numpy as np
import random
from matplotlib import pyplot as plt
%matplotlib inline
from time import sleep
from copy import deepcopy
import time
# keras
np.random.seed(13)
from keras.models import Sequential
from keras.layers import Dense, Embedding, Reshape, Activation, SimpleRNN, GRU, LSTM, Convolution1D, MaxPooling1D, Merge, Dropout
from IPython.display import SVG
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.datasets import imdb, reuters
from keras.preprocessing import sequence
from keras.optimizers import SGD, RMSprop

In [187]:
def right_click_on_square(coords):
    right_click_list.append(coords)

In [147]:
def click_on_square(coords):
    click_list.append(coords)

In [148]:
def check_adjacent(coords_1, coords_2):
    x1 = coords_1[0]
    x2 = coords_2[0]
    y1 = coords_1[1]
    y2 = coords_2[1]
    if abs(x1 - x2) <= 2 and abs(y1 - y2) <= 2:
        adj_1 = get_adjacent(currgrid, coords_1)
        adj_2 = get_adjacent(currgrid, coords_2)
        num_coords_1 = np.where(adj_1.reshape((3,3)) > 0)
        num_coords_1 = zip(map(lambda x: x - 1 + x1, num_coords_1[0]), map(lambda y: y - 1 + y1, num_coords_1[1]))
        num_coords_2 = np.where(adj_2.reshape((3,3)) > 0)
        num_coords_2 = zip(map(lambda x: x - 1 + x2, num_coords_2[0]), map(lambda y: y - 1 + y2, num_coords_2[1]))
        if len(set.intersection(set(num_coords_1), set(num_coords_2))) > 0:
            return True
        else:
            return False
    else:
        return False

In [149]:
def check_adjacent_list(coords_1, coords_list):
    for coord in coords_list:
        if check_adjacent(coords_1, coord):
            return True
    return False

In [150]:
def get_border_mysteries():
    border_squares = []
    mystery_squares = np.where(currgrid == -1)
    mystery_squares = zip(mystery_squares[0], mystery_squares[1])
    big_list = []
    mines_left = numberofmines - len(np.where(currgrid == -3)[0])
    
    for coords in mystery_squares:
        adjacents = get_adjacent(currgrid, coords)
        if sum(np.in1d(adjacents, np.arange(1,9))) > 0:
             border_squares.append(coords) 
    while len(border_squares) > 0:
        start_square = border_squares.pop(0)
        finding_stuff = True
        start_list = [start_square]
        while finding_stuff:
            i = 0
            finding_stuff = False
            while i < len(border_squares):
                sqr = border_squares[i]
                if check_adjacent_list(sqr, start_list):
                    start_list.append(border_squares.pop(i))
                    finding_stuff = True
                else:
                    i += 1
        big_list.append(start_list)
    return big_list

In [151]:
def get_adjacent(my_grid, coords):
    i = coords[0] + 1
    j = coords[1] + 1
    temp_grid = np.multiply(np.ones((my_grid.shape[0] + 2, my_grid.shape[1] + 2)),-10)
    temp_grid[1:my_grid.shape[0] + 1, 1:my_grid.shape[1] + 1] = deepcopy(my_grid)
    adj = np.concatenate((temp_grid[i-1][j-1:j+2],temp_grid[i][j-1:j+2],temp_grid[i+1][j-1:j+2]))
    return adj

In [152]:
def backtracker(lists, border_squares):
    list_of_lists = []
    found_a_branch = False
    if len(lists) == 0:

        lis_one = np.array([-2])
        lis_two = np.array([-4])
        if check_valid(lis_one, border_squares):
                list_of_lists.append(lis_one)
        if check_valid(lis_two, border_squares):
                list_of_lists.append(lis_two)  
    else:
        for arr in lists:
            lis_one = np.append(arr,-2)
            lis_two = np.append(arr,-4)
            if check_valid(lis_one, border_squares):
                list_of_lists.append(lis_one)
            if check_valid(lis_two, border_squares):
                list_of_lists.append(lis_two) 
    if len(lis_one) == len(border_squares) or len(list_of_lists) == 0:
        return list_of_lists
    else:
        return backtracker(list_of_lists, border_squares)     

In [153]:
def check_valid(arr, border_squares):
    temp_grid = deepcopy(currgrid)
    is_valid = True
    
    mines_left = numberofmines - len(np.where(temp_grid <= -3)[0])

    for i in range(len(arr)):
        temp_grid[border_squares[i]] = arr[i]
    coords = border_squares[len(arr) - 1]
    adjs = get_adjacent(temp_grid, coords)
    num_locs = np.where(adjs.reshape((3,3)) > 0)
    num_coords = zip(map(lambda x: x - 1 + coords[0],num_locs[0]), map(lambda y: y - 1 + coords[1], num_locs[1]))
    for coords_2 in num_coords:
        adjs_2 = get_adjacent(temp_grid, coords_2)
        bomb_count = sum(adjs_2 == -3) + sum(adjs_2 == -4)
        hypo_bomb_count = sum(adjs_2 == -4)
        mystery_count = sum(adjs_2 == -1)
        if bomb_count > adjs_2[4]:
            is_valid = False
            return is_valid
        if bomb_count < adjs_2[4] and mystery_count == 0:
            is_valid = False
            return is_valid
        if hypo_bomb_count > mines_left:
            is_valid = False
            return is_valid
    return is_valid

In [154]:
def check_for_commonalities(mine_list, border_squares):
    #print mine_list
    mine_list_rot = zip(*mine_list)
    did_something = False
    prob_list = []
    #print mine_list_rot
    for i in range(len(mine_list_rot)):
        possibilities = list(mine_list_rot[i])
        #print type(possibilities)
        #print possibilities
        if len(set(possibilities)) == 1 and list(set(possibilities))[0] == -4:
            right_click_on_square(border_squares[i])
            did_something = True
        if len(set(possibilities)) == 1 and list(set(possibilities))[0] == -2:
            click_on_square(border_squares[i])
            did_something = True
    return did_something#, prob_list

In [155]:
def choose_a_rando(mine_list, border_squares):
    #print mine_list
    mine_list_rot = zip(*mine_list)
    did_something = False
    prob_list = []
    mine_list_probs = []
    for possibilities in mine_list_rot:
        mine_list_probs.append(sum(np.array(possibilities) == -4))
    square_to_click = np.argmin(mine_list_probs)
    click_on_square(border_squares[square_to_click])

In [156]:
def brute_force_algorithm(start_square):
    border_squares = get_border_mysteries()
    something_done = False
    thing = False

    if len(border_squares) > 0:
        for border_cluster in border_squares:
            mine_list = backtracker([], border_cluster)
            thing = check_for_commonalities(mine_list, border_cluster)
            if thing:
                something_done = True
        if not something_done:
            flat_border_squares = sum(border_squares, [])
            mine_list = backtracker([], flat_border_squares)
            choose_a_rando(mine_list, flat_border_squares)
    else:
        click_on_square(start_square)
    
    return True

In [178]:
# """A command line version of Minesweeper"""
import random
import re
import time
import numpy as np
from string import ascii_lowercase
from collections import defaultdict
grids_and_bombs = list()
global games_count
games_count = 0 
global wins_count
wins_count = 0



def setupgrid(gridsize, start, numberofmines):
    emptygrid = np.zeros(gridsize)
    mines = getmines(emptygrid, start, numberofmines)

    for i, j in mines:
        emptygrid[i][j] = -10

    grid = getnumbers(emptygrid)

    return (grid, mines)


def showgrid(grid):
    gridsize = grid.shape

    horizontal = '   ' + (4 * gridsize[0] * '-') + '-'

    # Print top column letters
    toplabel = '     '

    for i in ascii_lowercase[:gridsize[1]]:
        toplabel = toplabel + i + '   '

    print(toplabel + '\n' + horizontal)

    # Print left row numbers
    for idx, i in enumerate(grid):
        row = '{0:2} |'.format(idx + 1)
        for j in i:
            row = row + ' ' + str(int(j)) + ' |'

        print(row + '\n' + horizontal)

    print('')


def getrandomcell(grid):
    gridsize = grid.shape

    a = random.randint(0, gridsize[0] - 1)
    b = random.randint(0, gridsize[1] - 1)

    return (a, b)


def getneighbors(grid, rowno, colno):
    gridsize = grid.shape
    neighbors = []

    for i in range(-1, 2):
        for j in range(-1, 2):
            if i == 0 and j == 0:
                continue
            elif -1 < (rowno + i) < gridsize[0] and -1 < (colno + j) < gridsize[1]:
                neighbors.append((rowno + i, colno + j))

    return neighbors


def getmines(grid, start, numberofmines):
    mines = []
    neighbors = getneighbors(grid, *start)

    for i in range(numberofmines):
        cell = getrandomcell(grid)
        while cell == start or cell in mines or cell in neighbors:
            cell = getrandomcell(grid)
        mines.append(cell)

    return mines


def getnumbers(grid):
    #print 'grid1'
    #print grid
    for rowno, row in enumerate(grid):
        for colno, cell in enumerate(row):
            if cell != -10:
                # Gets the values of the neighbors
                values = [grid[r][c] for r, c in getneighbors(grid,
                                                              rowno, colno)]

                # Counts how many are mines
                grid[rowno][colno] = values.count(-10)
    #print 'grid2'
    #print grid

    return grid


def showcells(grid, currgrid, rowno, colno):
    # Exit function if the cell was already shown
    if currgrid[rowno][colno] != -1:
        return

    # Show current cell
    currgrid[rowno][colno] = grid[rowno][colno]

    # Get the neighbors if the cell is empty
    if grid[rowno][colno] == 0:
        for r, c in getneighbors(grid, rowno, colno):
            # Repeat function for each neighbor that doesn't have a flag
            if currgrid[r][c] != -3:
                showcells(grid, currgrid, r, c)


def playagain():
    choice = raw_input('Play again? (y/n): ')
    return choice.lower() == 'y'


def parseinput(inputstring, gridsize, helpmessage):
    cell = ()
    flag = False
    message = "Invalid cell. " + helpmessage

    pattern = r'([a-{}])([0-9]+)(f?)'.format(ascii_lowercase[gridsize - 1])
    validinput = re.match(pattern, inputstring)

    if inputstring == 'help':
        message = helpmessage

    elif validinput:
        rowno = int(validinput.group(2)) - 1
        colno = ascii_lowercase.index(validinput.group(1))
        flag = bool(validinput.group(3))

        if -1 < rowno < gridsize:
            cell = (rowno, colno)
            message = ''

    return {'cell': cell, 'flag': flag, 'message': message}

def playgame():
    start_square = (5,5)
    gridsize = (9,9)
    global numberofmines
    numberofmines = 10
    #print grids_and_bombs
    global currgrid
    currgrid = np.negative(np.ones(gridsize))
    first_move = True
    #first_cell = tuple()
    grid = []
    flags = []
    starttime = 0

    global click_list
    click_list = []
    global right_click_list
    right_click_list = []
    currtime = time.time()
    while time.time() - currtime < 30:
        minesleft = numberofmines - len(flags)

        
        message = ''
        if len(click_list) > 0:

            cell = click_list.pop(0)
            flag = False
        elif len(right_click_list) > 0:

            cell = right_click_list.pop(0)
            flag = True
        else:
            thing = brute_force_algorithm(start_square)
            cell = False
            
        if cell:
            #print('\n\n')
            rowno, colno = cell
            currcell = currgrid[rowno][colno]

            if len(grid) ==0:
                grid, mines = setupgrid(gridsize, cell, numberofmines)
                #print 'got here'
            if not starttime:
                starttime = time.time()

            if flag:
                # Add a flag if the cell is empty
                if currcell == -1:
                    currgrid[rowno][colno] = -3
                    flags.append(cell)
                # Remove the flag if there is one
                elif currcell == -3:
                    currgrid[rowno][colno] = -1
                    flags.remove(cell)
                else:
                    message = 'Cannot put a flag there'

            # If there is a flag there, show a message
            elif cell in flags:
                message = 'There is a flag there'

            elif grid[rowno][colno] == -10:
                #print('Game Over\n')
                #showgrid(grid)
                if not first_move:
                    start_to_games_count[first_cell] +=1
#                     print first_cell
                    global games_count
                    games_count += 1
                    print wins_count/float(games_count)
                return

            elif currcell == -1:

                if first_move:
                    first_cell = cell
                    first_move = False
                
                showcells(grid, currgrid, rowno, colno)

            if set(flags) == set(mines):
                minutes, seconds = divmod(int(time.time() - starttime), 60)
                start_to_wins_count[first_cell] +=1
                start_to_games_count[first_cell] +=1
                #print first_cell
                global games_count
                games_count +=1
                global wins_count
                wins_count +=1


                return
       

<ipython-input-178-08d9ceb87fae>:275: SyntaxWarning: name 'games_count' is assigned to before global declaration
  global games_count
<ipython-input-178-08d9ceb87fae>:277: SyntaxWarning: name 'wins_count' is used prior to global declaration
  global wins_count


In [ ]:
for i in range(90000):
    playgame()

0.957640696976
0.95764493842
0.957649179015
0.957653418761
0.957657657658
0.957661895706
0.957666132906
0.957670369258
0.957674604763
0.95767883942
0.957683073229
0.957687306192
0.957691538308
0.957695769577
0.9576
0.957604239576
0.957608478304
0.957612716185
0.957616953219
0.95752123938
0.957525484709
0.95752972919
0.957533972822
0.957538215606
0.957542457542
0.957546698632
0.957550938873
0.957555178268
0.957559416816
0.957563654518
0.957567891374
0.957572127383
0.957576362547
0.957580596866
0.957584830339
0.957589062968
0.957593294752
0.957597525691
0.957601755786
0.957605985037
0.957610213445
0.957614441009
0.95761866773
0.957622893609
0.957627118644
0.957631342837
0.957635566188
0.957639788697
0.957644010365
0.957648231191
0.957652451176
0.95765667032
0.957660888623
0.957665106086
0.957669322709
0.957673538492
0.957677753436
0.95768196754
0.957686180804
0.95769039323
0.957694604818
0.957698815567
0.957703025478
0.957707234551
0.957711442786
0.957715650184
0.957719856745
0.957724062

In [95]:
3+3

6

In [ ]:
for cell,games in start_to_wins_count.items():
    if cell[0] < 0 or cell[1] < 0 or cell[0] > 8 or cell[1] > 8:
        print cell

In [ ]:
reflection_dict_games = defaultdict(int)
reflection_dict_wins = defaultdict(int)
reflection_dict_percent = defaultdict(int)

In [ ]:
for cell,games in start_to_wins_count.items():
    reflection_dict_wins[(abs(cell[0] - 7.5), abs(cell[1] - 7.5))] += games
for cell,wins in start_to_games_count.items():
    reflection_dict_games[(abs(cell[0] - 7.5), abs(cell[1] - 7.5))] += wins

In [ ]:
for first_cell, num in reflection_dict_wins.items():
    reflection_dict_percent[first_cell] = num/float(reflection_dict_games[first_cell])
    
    

In [ ]:
allgames_dict_percent = defaultdict()

In [ ]:
for first_cell, num in start_to_wins_count.items():
    allgames_dict_percent[first_cell] = num/float(start_to_games_count[first_cell])

In [ ]:
reflection_dict_games

In [ ]:
sorted(reflection_dict_percent.items(), key = lambda x: x[1], reverse = True)

In [ ]:
gridthing = np.zeros((8,8))

In [ ]:
for coords, perc in reflection_dict_percent.items():
    gridthing[int(coords[0] -.5)][int(coords[1] - .5)] = perc

In [ ]:
seaborn.heatmap(gridthing)

In [ ]:
import seaborn

In [ ]:
%matplotlib inline

In [ ]:
import pickle

In [ ]:
with open("couple_dicts.pkl", 'wb') as picklefile: 
     pickle.dump((start_to_wins_count, start_to_games_count), picklefile)

In [ ]:
seaborn.heatmap(grid_all)

In [ ]:
grid_all = np.zeros((16,16))

In [ ]:
for coords, perc in allgames_dict_percent.items():
    grid_all[int(coords[0])][int(coords[1])] = perc

In [ ]:
len(start_to_wins_count.items())

In [ ]:
start_to_win_percent = defaultdict(float)

In [ ]:
start_to_win_percent

In [ ]:
wins_count/float(games_count)

In [ ]:
a = [1,2]

In [ ]:
np.negative(np.ones((3,3)))


In [ ]:
bombs = thing[1]

In [ ]:
bomb_mat = np.array(bombs)

In [ ]:
grid_mat = np.array(grids)

In [ ]:
3+3

In [ ]:
bomb_mat = bomb_mat.astype(int)

In [ ]:
grid_mat[0].reshape(9,9)

In [ ]:
bomb_mat[0].reshape(9,9)

In [ ]:
model = Sequential()


model.add(Dense(output_dim=810,input_dim=81))
model.add(Activation("relu"))
model.add(Dense(output_dim=810,input_dim=810))
model.add(Activation("relu"))
model.add(Dense(output_dim=810,input_dim=810))
model.add(Activation("relu"))
model.add(Dense(output_dim=81,input_dim=810))


model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [ ]:
3 + 3

In [ ]:
model.fit(grid_mat, bomb_mat, nb_epoch=5, batch_size=32)

In [ ]:
just_a_grid = '[[-1. -1.  1.  0.  1. -1. -1. -1. -1.]\
 [-1. -1.  1.  0.  2. -1. -1. -1. -1.]\
 [-1. -1.  2.  0.  2. -1. -1. -1. -1.]\
 [-1. -1.  1.  0.  1.  2.  2.  1. -1.]\
 [ 1.  1.  1.  0.  0.  0.  0.  1. -1.]\
 [ 0.  0.  0.  0.  1.  1.  1.  1. -1.]\
 [ 0.  0.  0.  0.  1. -1. -1. -1. -1.]\
 [ 0.  0.  0.  0.  1.  1.  2. -1. -1.]\
 [ 0.  0.  0.  0.  0.  0.  1. -1. -1.]]'

just_a_grid = just_a_grid.replace('[', '').replace(']', '')

just_a_grid = just_a_grid.replace('.', '')

just_a_grid = just_a_grid.split(' ')

temp=[]
for x in just_a_grid:
    if x != '':
        temp.append(int(x))

just_a_grid = temp


just_a_grid = np.array(just_a_grid)

just_a_grid = just_a_grid.reshape((9,9))

just_a_grid = just_a_grid.reshape((1,81))

In [ ]:
just_a_grid

In [ ]:
just_a_grid.reshape((9,9))

In [ ]:
helo = model.predict(just_a_grid).reshape(9,9)

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
helo

In [ ]:
plt.imshow(helo, cmap='hot', interpolation='nearest')


In [ ]:
grid_mat.shape

In [ ]:
a = np.array([True, False])